In [31]:
import simplejson as simplejson
from bs4 import BeautifulSoup
import time
import requests
from random import randint
import urllib.parse
import json
import csv
from urllib.parse import unquote
import pandas as pd
import tqdm

USER_AGENT = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}



In [32]:
class SearchEngine:
    @staticmethod
    def search(query, sleep=True):
        if sleep:  # Prevents loading too many pages too soon
            time.sleep(randint(1, 20))
        temp_url = '+'.join(query.split())  # for adding + between words for the query
        url = 'http://www.search.yahoo.com/search?p=' + temp_url

        soup = BeautifulSoup(requests.get(url, headers=USER_AGENT).text, "html.parser")
#         print(soup.prettify())
#         print('\n'*10)
        new_results = SearchEngine.scrape_search_result(soup)
        return new_results

    @staticmethod
    def scrape_search_result(soup):
#         raw_results is page 1 result

        raw_results = soup.find_all("a", attrs={"class": "d-ib ls-05 fz-20 lh-26 td-hu tc va-bot mxw-100p"})
        
        page2 = soup.find("div",class_='compPagination')
#         print(page2.prettify())
        
        page2_url = page2.find("a",class_='next')
        page2_url = page2_url['href']
#         print(page2_url)
        
        page2_soup = BeautifulSoup(requests.get(page2_url, headers=USER_AGENT).text, "html.parser")
        
        page2_raw_results = page2_soup.find_all("a", attrs={"class": "d-ib ls-05 fz-20 lh-26 td-hu tc va-bot mxw-100p"})
        
#         print(len(raw_results))
        results = []
        count = 0
        # implement a check to get only 10 results and also check that URLs must not be duplicated
        
        all_results = raw_results+page2_raw_results
        
#         print(len(all_results))
        
        for anchor in all_results:
#             print(anchor)
            url = anchor['href']
#             print(url)
            start_idx = url.find('RU=')
            end_idx= url.find("/RK")
            
#             print(url[start_idx+3:end_idx])
            
            link = unquote(url[start_idx+3:end_idx])
            
            
            link = link.replace("https", "http")
            if link.endswith('/'):
                link = link[:-1]
#             link = link.lower()

            if link not in results:
                count = count + 1
                results.append(link)
            
            if count == 10:
                break
        
        return results


#############Driver code############

f = open("100QueriesSet2.txt", "r")
google_results = {}
with open('Google_Result2.json') as google:
    google_results = json.load(google)

# print(google_results)
data = {}
# queries is a counter query 1, query 2,..
queries = 0
avg_match = 0
avg_percentage = 0
avg_coeff = 0

statistics = []

list_of_dict = []

for line in tqdm.tqdm(f):
    
    queries = queries + 1
    print(queries)
    line = line.strip('\n')
    line = line.strip('\t')
    line = line.strip()
#     print(line)
#  res is a list
    res = SearchEngine.search(line)
#     number of links returned as a result.
#     print(len(res))

#     data[query] =[link1,..link10]

    data[line] = res
    google_links = google_results[line]
    clean_links = []
    
#     print(data[line])
#     print(google_results[line])
    
    for link in google_links:
        link = link.replace("https", "http")
        if link.endswith('/'):
            link = link[:-1]

#         link = link.lower()
        clean_links.append(link)
    
    
    match = 0
    d2 = 0
    idx = 0
    for link in res:
        if clean_links.count(link) > 0:
            match = match + 1
            d2 = d2 + pow(idx - clean_links.index(link), 2)
        idx = idx+1
    

    statline = []
    statline.append("Query " + str(queries))
    statline.append(match)
    statline.append(match * 10)
    
    avg_match = avg_match + match
    
    avg_percentage = avg_percentage + (match*10)
    
    
    if match == 0 or (match == 1 and d2 > 0):
        value=0
        statline.append(0)
#     d2 ==0 and match ==1
    elif match == 1:
        value=1
        statline.append(1)
        avg_coeff = avg_coeff + 1
    elif match > 1:
        value = 6*d2
        value = value/(match*(pow(match, 2)-1))
        value = 1 - value
        avg_coeff = avg_coeff + value
        statline.append(value)

    
    list_of_dict.append({'Queries':'Query '+str(queries),'Number of Overlapping Results':match,'Percent Overlap':match * 10,'Spearman Coefficient':value})
    # statline.append(d2)
    # print(statline)
    
    
    statistics.append(statline)

# print(statistics)
f.close()


0it [00:00, ?it/s]

1


1it [00:23, 23.18s/it]

2


2it [00:27, 11.94s/it]

...

98it [21:02, 15.35s/it]

99


99it [21:11, 13.33s/it]

100


100it [21:20, 12.81s/it]


In [33]:
json_data = json.dumps(data)

jsonDataFile = open("hw1.json", "w")
jsonDataFile.write(simplejson.dumps(simplejson.loads(json_data), indent=4, sort_keys=True))
jsonDataFile.close()

list_of_dict.append({'Queries':'Averages','Number of Overlapping Results':avg_match/100,'Percent Overlap':avg_percentage/100,'Spearman Coefficient':avg_coeff/100})
pd.DataFrame(list_of_dict).to_csv('hw1_jr.csv',index=False)

with open('hw1.csv', 'w', newline='') as csvfile:
    record_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    record_writer.writerow(['Queries', 'Number of Overlapping Results', 'Percent Overlap', 'Spearman Coefficient'])
    for stats in statistics:
        record_writer.writerow(stats)

    record_writer.writerow(['Averages', avg_match/100, avg_percentage/100, avg_coeff/100])

# print(json_data)
####################################

In [39]:
json_data = json.dumps(data)

jsonDataFile = open("hw1.json", "w")
jsonDataFile.write(simplejson.dumps(simplejson.loads(json_data), indent=4, sort_keys=False))
jsonDataFile.close()